In [2]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

import ssl
from urllib.error import URLError
from bs4 import BeautifulSoup as bs

## Extracting features of 2023 movies from Wikipedia

In [3]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2023"

In [4]:
#to disable SSL certificate


# Disable SSL certificate verification
context = ssl._create_unverified_context()

try:
    link = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2023'
    source = urllib.request.urlopen(link, context=context).read()
    soup = bs(source, 'lxml')
    # Process the BeautifulSoup object (soup) as needed
except URLError as e:
    print("URLError:", e)


In [5]:
tables = soup.find_all('table',class_='wikitable sortable')

In [6]:
len(tables)

4

In [7]:
type(tables[0])

bs4.element.Tag

In [8]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [9]:
dfs = [df1, df2, df3, df4]

In [10]:
df = pd.concat(dfs, ignore_index=True)

In [11]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,6,M3GAN,Universal Pictures / Blumhouse Productions / A...,Gerard Johnstone (director); Akela Cooper (scr...,[3]
1,J A N U A R Y,6,The Old Way,Saban Films / Saturn Films,Brett Donowho (director); Carl W. Lucas (scree...,[4]
2,J A N U A R Y,11,The Devil Conspiracy,Samuel Goldwyn Films,Nathan Frankowski (director); Ed Alan (screenp...,[5]
3,J A N U A R Y,13,Plane,Lionsgate / MadRiver Pictures / Di Bonaventura...,Jean-François Richet (director); Charles Cummi...,[6]
4,J A N U A R Y,13,House Party,Warner Bros. Pictures / New Line Cinema,"Calmatic (director); Jamal Olori, Stephen Glov...",[7]
...,...,...,...,...,...,...
324,D E C E M B E R,22,Memory,Ketchup Entertainment / Mubi,Michel Franco (director/screenplay); Jessica C...,[311]
325,D E C E M B E R,25,The Color Purple,Warner Bros. Pictures / Harpo Films / Amblin E...,"Blitz Bazawule (director), Marcus Gardley (scr...",[312]
326,D E C E M B E R,25,The Boys in the Boat,Metro-Goldwyn-Mayer / Smokehouse Pictures,"George Clooney (director), Mark L. Smith (scre...",[313]
327,D E C E M B E R,25,Ferrari,Neon / STXfilms,"Michael Mann (director), Troy Kennedy Martin (...",[314]


In [12]:
df_2023 = df[['Title','Cast and crew']]

In [13]:
df_2020

NameError: name 'df_2020' is not defined

In [14]:
!pip install tmdbv3api

In [15]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'b1839aa17a5fded5e7a15224d03be724'

In [16]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [17]:
df_2023['genres'] = df_2023['Title'].map(lambda x: get_genre(str(x)))

C:\Users\Horizon\AppData\Local\Temp\ipykernel_7000\2917389152.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['genres'] = df_2023['Title'].map(lambda x: get_genre(str(x)))


In [68]:
df_2023

,Title,Cast and crew,genres
0,M3GAN,Gerard Johnstone (director); Akela Cooper (scr...,Science Fiction Horror
1,The Old Way,Brett Donowho (director); Carl W. Lucas (scree...,Western
2,The Devil Conspiracy,Nathan Frankowski (director); Ed Alan (screenp...,Horror Fantasy Science Fiction Thriller
3,Plane,Jean-François Richet (director); Charles Cummi...,Action Adventure Thriller
4,House Party,"Calmatic (director); Jamal Olori, Stephen Glov...",Comedy
...,...,...,...
322,The Iron Claw,"Sean Durkin (director/screenplay); Zac Efron, ...",Drama
323,Memory,Michel Franco (director/screenplay); Jessica C...,Action Thriller Crime
324,The Color Purple,"Blitz Bazawule (director), Marcus Gardley (scr...",Drama
325,The Boys in the Boat,"George Clooney (director), Mark L. Smith (scre...",Drama History


In [69]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [71]:
df_2023['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\Horizon\AppData\Local\Temp\ipykernel_7572\1289979380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))


In [72]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [73]:
df_2023['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\Horizon\AppData\Local\Temp\ipykernel_7572\1787722408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [74]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [75]:
df_2023['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [76]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [78]:
df_2023['actor_3_name'] = df_2023['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [79]:
df_2023

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,M3GAN,Gerard Johnstone (director); Akela Cooper (scr...,Science Fiction Horror,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald
1,The Old Way,Brett Donowho (director); Carl W. Lucas (scree...,Western,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,NaN
2,The Devil Conspiracy,Nathan Frankowski (director); Ed Alan (screenp...,Horror Fantasy Science Fiction Thriller,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall
3,Plane,Jean-François Richet (director); Charles Cummi...,Action Adventure Thriller,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An
4,House Party,"Calmatic (director); Jamal Olori, Stephen Glov...",Comedy,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom
...,...,...,...,...,...,...,...
322,The Iron Claw,"Sean Durkin (director/screenplay); Zac Efron, ...",Drama,Sean Durkin,Zac Efron,Jeremy Allen White,Harris Dickinson
323,Memory,Michel Franco (director/screenplay); Jessica C...,Action Thriller Crime,Michel Franco,Jessica Chastain,Peter Sarsgaard,Merritt Wever
324,The Color Purple,"Blitz Bazawule (director), Marcus Gardley (scr...",Drama,Blitz Bazawule,Fantasia Barrino,Taraji P. Henson,Danielle Brooks
325,The Boys in the Boat,"George Clooney (director), Mark L. Smith (scre...",Drama History,George Clooney,Callum Turner,Joel Edgerton,Peter Guinness


In [80]:
df_2023 = df_2023.rename(columns={'Title':'movie_title'})

In [81]:
new_df23 = df_2023.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [82]:
new_df23

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror,M3GAN
1,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,NaN,Western,The Old Way
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,The Devil Conspiracy
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action Adventure Thriller,Plane
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,House Party
...,...,...,...,...,...,...
322,Sean Durkin,Zac Efron,Jeremy Allen White,Harris Dickinson,Drama,The Iron Claw
323,Michel Franco,Jessica Chastain,Peter Sarsgaard,Merritt Wever,Action Thriller Crime,Memory
324,Blitz Bazawule,Fantasia Barrino,Taraji P. Henson,Danielle Brooks,Drama,The Color Purple
325,George Clooney,Callum Turner,Joel Edgerton,Peter Guinness,Drama History,The Boys in the Boat


In [83]:
new_df23['comb'] = new_df23['actor_1_name'] + ' ' + new_df23['actor_2_name'] + ' '+ new_df23['actor_3_name'] + ' '+ new_df23['director_name'] +' ' + new_df23['genres']

In [84]:
new_df23.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      3
actor_3_name     21
genres            0
movie_title       0
comb             21
dtype: int64

In [85]:
new_df23 = new_df23.dropna(how='any')

In [86]:
new_df23.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [87]:
new_df23['movie_title'] = new_df23['movie_title'].str.lower()

C:\Users\Horizon\AppData\Local\Temp\ipykernel_7572\3055323516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df23['movie_title'] = new_df23['movie_title'].str.lower()


In [88]:
new_df23

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror,m3gan,Allison Williams Violet McGraw Amie Donald Ger...
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,the devil conspiracy,Alice Orr-Ewing Joe Doyle Eveline Hall Nathan ...
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action Adventure Thriller,plane,Gerard Butler Mike Colter Yoson An Jean-Franço...
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,house party,Tosin Cole Jacob Latimore Karen Obilom Calmati...
5,John Hyams,Gideon Adlon,Bethlehem Million,Marc Menchaca,Horror Thriller Mystery,sick,Gideon Adlon Bethlehem Million Marc Menchaca J...
...,...,...,...,...,...,...,...
322,Sean Durkin,Zac Efron,Jeremy Allen White,Harris Dickinson,Drama,the iron claw,Zac Efron Jeremy Allen White Harris Dickinson ...
323,Michel Franco,Jessica Chastain,Peter Sarsgaard,Merritt Wever,Action Thriller Crime,memory,Jessica Chastain Peter Sarsgaard Merritt Wever...
324,Blitz Bazawule,Fantasia Barrino,Taraji P. Henson,Danielle Brooks,Drama,the color purple,Fantasia Barrino Taraji P. Henson Danielle Bro...
325,George Clooney,Callum Turner,Joel Edgerton,Peter Guinness,Drama History,the boys in the boat,Callum Turner Joel Edgerton Peter Guinness Geo...


In [1]:
old_df = pd.read_csv('final_data.csv')

NameError: name 'pd' is not defined

In [ ]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5864,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5865,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5866,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...
5867,Chinonye Chukwu,Alfre Woodard,Wendell Pierce,Aldis Hodge,Drama,clemency,Alfre Woodard Wendell Pierce Aldis Hodge Chino...


In [ ]:
final_df = old_df.append(new_df20,ignore_index=True)

In [ ]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6005,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly,Action Drama,top gun: maverick,Tom Cruise Miles Teller Jennifer Connelly Jose...
6006,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Adventure Family,the croods 2,Nicolas Cage Emma Stone Ryan Reynolds Joel Cra...
6007,Liesl Tommy,Jennifer Hudson,Forest Whitaker,Marlon Wayans,Music Drama,respect,Jennifer Hudson Forest Whitaker Marlon Wayans ...
6008,Ridley Scott,Matt Damon,Adam Driver,Jodie Comer,Drama,the last duel,Matt Damon Adam Driver Jodie Comer Ridley Scot...


In [ ]:
final_df.to_csv('main_data.csv',index=False)